# 1. Getting Started with Tensorflow

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 03/09/2024   | Martin | Created   | Started chapter 1 | 

# Content

* [Introduction](#introduction)

# Introduction

General workflow of Tensorflow:

1. Import or generate datasets
2. Transform and normalize data
3. Partition datasets into training, test and validation sets
4. Set algorithm parameters (hyperparameters)
5. Initialize variables
6. Define the model structure
7. Declare loss function
8. Initialise and train the model
9. Evaluate the model
10. Tune hyper parameters
11. Deploy/ predict new outcomes

In [13]:
# 1.
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

data = tfds.load("iris", split='train')

# 4.
epochs = 1000
batch_size = 32
input_size = 4
output_size = 3
learning_rate = 0.001

# 5.
weights = tf.Variable(tf.random.normal(
  shape=(input_size, output_size),
  dtype=tf.float32
))
biases = tf.Variable(tf.random.normal(
  shape=(output_size,),
  dtype=tf.float32
))

# 8.
optimizer = tf.optimizers.SGD(learning_rate)

for _ in range(epochs):
  # 2.
  for batch in data.batch(batch_size, drop_remainder=True):
    labels = tf.one_hot(batch['label'], 3)
    X = batch['features']
    X = (X - np.mean(X) / np.std(X))

    with tf.GradientTape() as tape:
      # 6.
      logits = tf.add(tf.matmul(X, weights), biases) # logistic regression (actually is linear regression first)

      # 7.
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels, logits)
      )

    # 8.
    gradients = tape.gradient(loss, [weights, biases])
    optimizer.apply_gradients(zip(gradients, [weights, biases]))

print(f"final loss is: {loss.numpy():.3f}")
preds = tf.math.argmax(tf.add(tf.matmul(X, weights), biases), axis=1)
ground_truth = tf.math.argmax(labels, axis=1)
for y_true, y_pred in zip(ground_truth.numpy(), preds.numpy()):
  print(f"real label: {y_true} | fitted: {y_pred}")

final loss is: 0.313
real label: 0 | fitted: 0
real label: 1 | fitted: 1
real label: 1 | fitted: 2
real label: 2 | fitted: 1
real label: 2 | fitted: 2
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 1 | fitted: 1
real label: 2 | fitted: 2
real label: 1 | fitted: 2
real label: 0 | fitted: 0
real label: 1 | fitted: 1
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 1 | fitted: 1
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 1 | fitted: 1
real label: 0 | fitted: 0
real label: 0 | fitted: 0
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 1 | fitted: 2
real label: 2 | fitted: 2
real label: 0 | fitted: 0
real label: 2 | fitted: 2
